In [1]:
#!/usr/bin/env python3
# Copyright 2022 ETH Zurich and University of Bologna.
# Licensed under the Apache License, Version 2.0, see LICENSE for details.
# SPDX-License-Identifier: Apache-2.0

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import argparse
import pathlib
import hjson
import random
import os

In [2]:
np.random.seed(42)
torch.manual_seed(42)
global verbose

In [3]:
def array_to_cstr(a):
    out = '{'
    if isinstance(a, np.ndarray):
        a = a.flat
    if isinstance(a, torch.Tensor):
        a = a.numpy().flat
    for el in a:
        out += '{}, '.format(el)
    out = out[:-2] + '}'
    return out

In [ ]:
# TODO: check what is missing for CNN
def emit_mnist_data(name='mnist_cnn', **kwargs):
    
    # constants
    IN_CH1 = kwargs['IN_CH1']
    IN_CH2 = kwargs['IN_CH2']
    OUT_CH = kwargs['OUT_CH']
    DATASET_SIZE = kwargs['DATASET_SIZE']
    
    # data
    MAT_INPUT = kwargs['INPUT']
    MAT_LABELS = kwargs['LABELS']

    # network init parameters from golden model
    MAT_WEIGHTS = kwargs['WEIGHTS']
    MAT_BIASES = kwargs['BIASES']
    MAT_WEIGHT_GRADS = kwargs['WEIGHT GRADIENTS']
    MAT_BIAS_GRADS = kwargs['BIAS GRADIENTS'] 

    IN_CH = IN_CH1*IN_CH2
    
    layer_str = ''
    layer_str += '#include "network.h"\n\n'
    layer_str += f'network_t {name}_t = {{\n'
    layer_str += f'\t.IN_CH1 = {IN_CH1},\n'
    layer_str += f'\t.IN_CH2 = {IN_CH2},\n'
    layer_str += f'\t.OUT_CH = {OUT_CH},\n'
    layer_str += f'\t.dtype = FP{kwargs["prec"]}\n'
    layer_str += '};\n\n\n'

    ctypes = {
        '64': 'double',
        '32': 'float',
        '16': '__fp16',
        'B16': '__bf16',
        '8': 'char'
    }

    dtype = ctypes[str(kwargs['prec'])]

    # network initialization
    layer_str += f'static {dtype} {name}_weights_dram [{OUT_CH}][{IN_CH}] = ' + array_to_cstr(MAT_WEIGHTS) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_biases_dram [{OUT_CH}][{1}] = ' + array_to_cstr(MAT_BIASES) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_weight_grads_dram [{OUT_CH}][{IN_CH}] = ' + array_to_cstr(MAT_WEIGHT_GRADS) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_bias_grads_dram [{OUT_CH}][{1}] = ' + array_to_cstr(MAT_BIAS_GRADS) + ';\n\n\n'


    # input data
    layer_str += f'static {dtype} {name}_images_dram [{DATASET_SIZE*IN_CH}][{1}] = ' + array_to_cstr(MAT_INPUT) + ';\n\n\n'
    layer_str += f'static uint32_t {name}_labels_dram [{DATASET_SIZE}][{1}] = ' + array_to_cstr(MAT_LABELS) + ';\n\n\n'
    #layer_str += f'static {dtype} {name}_images_dram [{IN_CH}][{1}] = ' + array_to_cstr(MAT_INPUT) + ';\n\n\n'
    #layer_str += f'static uint32_t {name}_labels_dram[{1}] = ' + array_to_cstr(MAT_LABELS) + ';\n\n\n'

    return layer_str


In [4]:
# download MNIST dataset using DataLoader

transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
mnist_dataset = MNIST(PATH_DATASETS, train=True, transform=transform, download=True)

# set seeds for reproducability 
g = torch.Generator()
g.manual_seed(42)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

mnist_dl = DataLoader(mnist_dataset, worker_init_fn=seed_worker, generator=g)

In [5]:
print(mnist_dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: .
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )


In [46]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)
        print(x.shape)       
        output = self.out(x)
        print(output)
        return output, x    # return x for visualization

In [47]:
net = CNN()
print(net)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [9]:
"""
Now we iterate through a smaller subset of the dataset 
to retrieve the image data with their
respective labels
"""

data_iterator = iter(mnist_dl)

for i in range(0, 5):
    s_image, s_label = data_iterator.next()
    np_s_image = s_image.numpy().flatten()
    np_s_label = s_label.numpy().flatten()
    if(i==0):
        s_images = np.array(np_s_image.tolist())
        s_labels = np.array(np_s_label.tolist())
    else:
        s_images = np.append(s_images, np_s_image)
        s_labels = np.append(s_labels, np_s_label)

In [10]:
first_im, first_label = next(iter(mnist_dl))

In [48]:
torch.manual_seed(42)

weights_conv1 = net.conv1[0].weight
biases_conv1 = net.conv1[0].bias
weights_conv2 = net.conv2[0].weight
biases_conv2 = net.conv2[0].bias

criterion = nn.CrossEntropyLoss()

for i in range(1):
    net.zero_grad
    output = net(first_im)[0]
    loss = criterion(output, first_label)
    loss.backward()
    weight_grads_conv1 = net.conv1[0].weight.grad
    bias_grads_conv1 = net.conv1[0].bias.grad
    weight_grads_conv2 = net.conv2[0].weight.grad
    bias_grads_conv2 = net.conv2[0].bias.grad


torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 14, 14])
torch.Size([1, 32, 7, 7])
torch.Size([1, 1568])
tensor([[ 0.0022,  0.0393, -0.0614, -0.1077,  0.0828,  0.0614, -0.0228,  0.0396,
          0.1014, -0.0500]], grad_fn=<AddmmBackward0>)


In [66]:
print(first_im.shape)
net.conv1[0](first_im).shape

torch.Size([1, 1, 28, 28])


torch.Size([1, 16, 28, 28])

In [68]:
print(weights_conv1.shape)
print(biases_conv1.shape)

torch.Size([16, 1, 5, 5])
torch.Size([16])


In [70]:
weights_conv1[0].shape

torch.Size([1, 5, 5])

In [30]:
from torchvision import models
from torchsummary import summary
summary(net, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             416
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]          12,832
              ReLU-5           [-1, 32, 14, 14]               0
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Linear-7                   [-1, 10]          15,690
Total params: 28,938
Trainable params: 28,938
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.32
Params size (MB): 0.11
Estimated Total Size (MB): 0.44
----------------------------------------------------------------


In [71]:
def calculate_target_size(img_size: int, kernel_size: int) -> int:
    num_pixels = 0
    
    # From 0 up to img size (if img size = 224, then up to 223)
    for i in range(img_size):
        # Add the kernel size (let's say 3) to the current i
        added = i + kernel_size
        # It must be lower than the image size
        if added <= img_size:
            # Increment if so
            num_pixels += 1
            
    return num_pixels

In [89]:
calculate_target_size(img_size=32, kernel_size=5)

28

In [100]:
def convolve(img: np.array, kernel: np.array) -> np.array:
    # Assuming a rectangular image
    tgt_size = calculate_target_size(
        img_size=img.shape[0],
        kernel_size=kernel.shape[0]
    )
    # To simplify things
    k = kernel.shape[0]
    
    # 2D array of zeros
    convolved_img = np.zeros(shape=(tgt_size, tgt_size))
    
    # Iterate over the rows
    for i in range(tgt_size):
        # Iterate over the columns
        for j in range(tgt_size):
            # img[i, j] = individual pixel value
            # Get the current matrix
            mat = img[i:i+k, j:j+k]
            # Apply the convolution - element-wise multiplication and summation of the result
            # Store the result to i-th row and j-th column of our convolved_img array
            convolved_img[i, j] = np.sum(np.multiply(mat, kernel))
            
    return convolved_img

In [79]:
first_im[0][0].shape

torch.Size([28, 28])

In [82]:
first_im[0][0].numpy().shape

(28, 28)

In [85]:
first_conv_img = convolve(first_im[0][0].numpy(), weights_conv1[0].detach().numpy())
first_conv_img.shape

(28, 28)

In [88]:
np.pad(first_im[0][0].numpy(), ((2,2), (2,2)), 'constant', constant_values=0).shape

(32, 32)

In [101]:
first_conv_img = convolve(np.pad(first_im[0][0].numpy(), ((2,2), (2,2)), 'constant', constant_values=0), weights_conv1[0].detach().numpy())
first_conv_img.shape

(32, 32)

In [120]:
np.set_printoptions(threshold=sys.maxsize)
print(first_conv_img)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0

In [122]:
biases_conv1[0].detach().numpy()

array(-0.05595953, dtype=float32)

In [123]:
first_conv_img + biases_conv1[0].detach().numpy()

array([[-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953],
       [-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
        -0.05595953, -0.05595953],
       [-0.05595953, -0.05595953, -0.05595953, -0.0559

In [111]:
biases_conv1.detach().numpy().shape[0]

16

In [110]:
biases_conv1.detach().numpy()

array([-0.05595953,  0.13507354,  0.01593194,  0.00902367, -0.04922011,
       -0.18111794, -0.18805149, -0.09560301, -0.10166428,  0.06231072,
       -0.05822215, -0.07824443,  0.19068597,  0.06966458,  0.14258046,
       -0.09682255], dtype=float32)

In [95]:
net.conv1[0](first_im)[0][0].shape

torch.Size([28, 28])

In [129]:
net.conv1[0](first_im)[0][0].detach().numpy()[3]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05781687, -0.06554271, -0.05869401, -0.06374829, -0.13307634,
       -0.08849649, -0.14184302, -0.04242297, -0.25899282, -0.20098922,
       -0.1947165 , -0.11197067, -0.15200649, -0.21128216, -0.15689726,
       -0.10185251, -0.05595953, -0.05595953], dtype=float32)

In [131]:
(first_conv_img + biases_conv1[0].detach().numpy())[4]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953])

In [133]:
first_im[0][0].shape

torch.Size([28, 28])

In [145]:
# image dimensions
w_in = first_im[0][0].numpy().shape[0]
h_in = first_im[0][0].numpy().shape[1]
# input channel(s)
c_in = 1
print("Input Feature Map: {}x{}x{}".format(w_in, h_in, c_in))
# output channels
c_out = 16
# kernel size
k = 5
# stride
s = 1
# padding
p = 2
# output dimensions
w_out = (w_in - k + 2 * p) // s + 1
h_out = (h_in - k + 2 * p) // s + 1
print("Output Feature Map: {}x{}x{}".format(w_out, h_out, c_out))

Input Feature Map: 28x28x1
Output Feature Map: 28x28x16


In [172]:
# pad the image data with zeros
# pad_width = ((2,2), (2,2))
# first_im[0][0].numpy() = np.pad(first_im[0][0].numpy(), pad_width, 'constant', constant_values=0)
# img_padded = np.pad(first_im[0][0].numpy(), ((2,2), (2,2)), 'constant', constant_values=0)
# img_padded.shape
img_padded = F.pad(first_im, (2, 2, 2, 2))
img_padded.shape

torch.Size([1, 1, 32, 32])

In [175]:
result = np.zeros(shape=(1, c_out, w_out, h_out))
for o1 in range(w_out):
    for o2 in range(h_out):
        for co in range(c_out):
            total = 0
            for ci in range(c_in):
                kt = 0
                for kh in range(k):
                    for kw in range(k):
                        weight = weights_conv1[co][ci][kh][kw].detach().numpy()
                        pos1 = kh + o1 * s
                        pos2 = kw + o2 * s
                        value = img_padded[0][ci][pos1][pos2].numpy()
                        kt += weight * value
                total += kt
            result[0][co][o1][o2] = total + biases_conv1[co].detach().numpy()


In [177]:
result.shape

(1, 16, 28, 28)

In [179]:
net.conv1[0](first_im).shape

torch.Size([1, 16, 28, 28])

In [181]:
result[0][0][3]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05781688, -0.06554271, -0.05869401, -0.06374828, -0.13307635,
       -0.08849649, -0.14184302, -0.04242296, -0.25899283, -0.20098922,
       -0.19471649, -0.11197067, -0.15200648, -0.21128216, -0.15689726,
       -0.10185252, -0.05595953, -0.05595953])

In [178]:
net.conv1[0](first_im)[0][0].detach().numpy()[3]

array([-0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05595953, -0.05595953, -0.05595953, -0.05595953, -0.05595953,
       -0.05781687, -0.06554271, -0.05869401, -0.06374829, -0.13307634,
       -0.08849649, -0.14184302, -0.04242297, -0.25899282, -0.20098922,
       -0.1947165 , -0.11197067, -0.15200649, -0.21128216, -0.15689726,
       -0.10185251, -0.05595953, -0.05595953], dtype=float32)